## Model 2
We have one row per delivery hour where our target is the best price that occurs later

## Imports

In [2]:
import pandas as pd
import numpy as np
import ast
import pytz
import re
import matplotlib.pyplot as plt
import xgboost as xgb 
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report

import warnings
warnings.filterwarnings("ignore")

## Read file

In [23]:
df = pd.read_parquet('Data/formatted_orderbook.parquet')

In [5]:
df.columns

Index(['contractId', 'contractName', 'dlvryStart', 'dlvryEnd', 'dlvryAreaId',
       'marketId', 'area', 'recorded', 'time', 'orderId_bid', 'orderId_ask',
       'orderPrice_bid', 'orderPrice_ask', 'orderQty_bid', 'orderQty_ask',
       'DA_price', 'Delivery Start (UTC)', 'Delivery End (UTC)', 'dlvryMonth',
       'dlvryDay', 'dlvryHour', 'dlvryWeekday', 'timeMonth', 'timeDay',
       'timeHour', 'timeWeekday', 'timeMinute', 'orderPrice_bid_best',
       'orderQty_bid_best', 'orderPrice_ask_best', 'orderQty_ask_best',
       'exists_bid', 'exists_ask', 'best_far_bid', 'best_far_ask',
       'better_bid_later', 'better_ask_later', 'future_best_bid',
       'future_best_ask'],
      dtype='object')

## Formatting

In [22]:
def onlyKeepTheRowThatIsSixHoursBeforeDelivery(df):
    # First make a copy to avoid modifying the original dataframe
    filtered_df = df.copy()
    result = pd.DataFrame()
    for dlvry_time, group in filtered_df.groupby('dlvryStart'):
        # Calculate the time that is six hours before delivery
        target_time = dlvry_time- pd.Timedelta(hours=6)
        row = filtered_df[(filtered_df['time'] == target_time) & (filtered_df['dlvryStart'] == dlvry_time)]
        result = pd.concat([result, row]) 

    return result

In [20]:
df = onlyKeepTheRowThatIsSixHoursBeforeDelivery(df)

In [21]:
df.head()

""
